In [1]:
# Read the prduct reviews data from the Lakehouse

product_reviews_path="abfss://Electroniz_Workspace@onelake.dfs.fabric.microsoft.com/Electroniz_Lakehouse.Lakehouse/Files/bronze/social/Electronics.txt.gz"
df_product_reviews=spark.read.csv(product_reviews_path).withColumnRenamed('_c0','review')
display(df_product_reviews)

StatementMeta(, e7163e41-66fe-4259-8fc4-fe52a225b488, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 46bc55a4-eaca-4660-899a-cd45cbca351a)

In [2]:
# Code generated by Data Wrangler for PySpark DataFrame

def clean_data(df_product_reviews):
    # Filter rows based on column: 'review'
    df_product_reviews = df_product_reviews.filter((df_product_reviews['review'].startswith("review/text")) | (df_product_reviews['review'].startswith("review/summary")))
    return df_product_reviews

df_product_reviews_clean = clean_data(df_product_reviews)
display(df_product_reviews_clean)

StatementMeta(, e7163e41-66fe-4259-8fc4-fe52a225b488, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9ba564f6-399f-4960-b150-8fc281b8e5cd)

In [3]:
# Run sentiment analysis 

import synapse.ml.core
from synapse.ml.services.language import AnalyzeText
from pyspark.sql.functions import col

model = (AnalyzeText()
        .setTextCol("review")
        .setKind("SentimentAnalysis")
        .setOutputCol("response"))


#  Apply the model to the input Data Frame and
#  Extract the sentiment result:

result = model.transform(df_product_reviews_clean)\
        .withColumn("documents", col("response.documents"))\
        .withColumn("sentiment", col("documents.sentiment"))

display (result.select("review", "sentiment"))

StatementMeta(, e7163e41-66fe-4259-8fc4-fe52a225b488, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6cd3d0a7-19be-45ca-bce2-f55d586063dd)

In [5]:
# Code generated by Data Wrangler for PySpark DataFrame

from pyspark.sql import functions as F
from pyspark.sql import types as T

def clean_data(df_product_reviews):
    # Filter rows based on column: 'review'
    df_product_reviews = df_product_reviews.filter(df_product_reviews['review'].startswith("review/score"))
    # Split text using string ':' in column: 'review'
    split_col = F.split(df_product_reviews['review'], ':')
    max_size = df_product_reviews.select(F.max(F.size(split_col))).collect()[0][0]
    old_cols = df_product_reviews.columns
    new_cols = []
    loc_0 = df_product_reviews.columns.index('review')
    for i in range(max_size):
        cur_col_name = 'review_%d' % i
        new_cols.append(cur_col_name)
        df_product_reviews = df_product_reviews.withColumn(cur_col_name, split_col.getItem(i))
    df_product_reviews = df_product_reviews.select(*old_cols[:loc_0], *new_cols, *old_cols[loc_0+1:])
    # Rename column 'review_0' to 'review'
    df_product_reviews = df_product_reviews.withColumnRenamed('review_0', 'review')
    # Rename column 'review_1' to 'score'
    df_product_reviews = df_product_reviews.withColumnRenamed('review_1', 'score')
    # Change column type to float16 for column: 'score'
    # ⚠️ Could not convert to the specified bits for type
    df_product_reviews = df_product_reviews.withColumn('score', df_product_reviews['score'].cast(T.FloatType()))
    return df_product_reviews

df_product_reviews_clean = clean_data(df_product_reviews)
display(df_product_reviews_clean)

StatementMeta(, e7163e41-66fe-4259-8fc4-fe52a225b488, 105, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3f2eac5a-ed66-4559-9f36-73aab4b8c30a)